In [38]:
import os
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta, datetime
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp

import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp



%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc## read in Nes from BEAST log files

import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [20]:
## read in Nes from BEAST log files
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [21]:
# make a new Ne dataframe that summarizes the 95% HPD estimate with mean for each interval 
def generate_ne_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            interval = i.split(".")[1]
            local_series = input_df[i].astype('float').to_numpy()
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
        
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [22]:
# read in migration rates
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])           
                
                
    return(mig_rates_dict)

In [23]:
# make a new migration dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_mig_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "immigrationRate." in i:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)

            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_mig_log":mean_log,"mean_mig_linear":mean_linear, 
                                                   "median_mig_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
   
            

            
    return(new_df)

In [79]:
def plot_ne_and_intro_comparison(log_file):
    
    ## plotting parameters
    width_altair = 300
    height_altair = 250
    
    ## read in log file for Ne extraction
    Ne_skyline = read_in_Ne_changes_mascot(log_file)

    #make pandas dataframe
    Ne_df = pd.DataFrame.from_dict(Ne_skyline)
    
    #remove burnin
    burnin_percent = 0.3
    print(len(Ne_df))
    rows_to_remove = int(len(Ne_df)* burnin_percent) ## remove burnin
    Ne_df = Ne_df.iloc[rows_to_remove:]

    print(len(Ne_df))
    Ne_df = Ne_df.reset_index()
    
    #create summary dataframe for easier plotting
    ne_summary = generate_ne_summary_df(Ne_df)
    
    #calculate dates and reverse time since phylo is backwards in time
    ne_summary['days'] = (ne_summary.interval.astype(int))*7
    ne_summary['date'] = dt.strptime("2024-12-12",  "%Y-%m-%d") - ne_summary.days.map(timedelta) ## this is the date of the most recent sample
    ne_summary = ne_summary.reset_index()

    #calculate three week rolling mean
    columns = ["mean_Ne_linear", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
    for column in columns:
        # Create a new column for the moving average
        ne_summary[f'{column}_MA'] = ne_summary[column].rolling(3, min_periods =1).mean()
        
    
    #create Ne plots
    line_ne = alt.Chart(ne_summary).mark_area(interpolate='monotone', opacity = 1, clip = True, color = "#7c5981").encode(
        alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
        alt.Y('lower_hpd_linear_50_MA',axis=alt.Axis(title="Effective Pop. Size(magenta)", grid=False),scale=alt.Scale(domain=(0, 60))),
        alt.Y2('upper_hpd_linear_50_MA' ),
    ).properties(
        width=450,
        height=150)

    band_ne = alt.Chart(ne_summary).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "#7c5981").encode(
        alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
        alt.Y('lower_hpd_linear_MA',axis=alt.Axis(title="Effective Pop. Size(magenta)", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
        alt.Y2('upper_hpd_linear_MA' ),
    ).properties(
        width=450,
        height=150
    )
        
    # read in mpox case data
    la_mpox_cases_df = pd.read_csv("../multitree_coalescent/data/monkeypox_data.csv")
    la_mpox_cases_df = la_mpox_cases_df.dropna(how = "all").dropna(axis = "columns", how = "all")
    la_mpox_cases_df = la_mpox_cases_df.rename(columns= {"Unnamed: 0": "date"}); la_mpox_cases_df.head()
    
    
    #plot mpox cases
    cases = alt.Chart(la_mpox_cases_df).mark_bar(interpolate='monotone', opacity = 0.5, size = 1).encode(
    alt.X('date:T', axis=alt.Axis(title="", tickCount = "month",grid=False, format="%B %Y")),
    alt.Y('cases',axis=alt.Axis(title="Cases (Blue)", grid=False))
    ).properties(
        width=450,
        height=150
    )
    
    #combine and save plots
    ne_case_plot = (cases + line_ne).resolve_scale(y = "independent")
    
    ### now do the same but for the rate of introduction!
    migration_rates_f = read_in_forward_migration_rates_mascot(log_file)
    mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

    #remove burnin
    burnin_percent = 0.3
    print(len(mig_df_f))
    rows_to_remove = int(len(mig_df_f)* burnin_percent)
    mig_df_f = mig_df_f.iloc[rows_to_remove:]

    print(len(mig_df_f))
    mig_df_f = mig_df_f.reset_index()
    
    #generate summary df
    mig_summary = generate_mig_summary_df(mig_df_f)
    
    #format dates
    mig_summary['days'] = (mig_summary.interval.astype(int))*7
    mig_summary['date'] = dt.strptime("2024-12-12",  "%Y-%m-%d") - mig_summary.days.map(timedelta)
    
    # plot it!
    line_mig = alt.Chart(mig_summary).mark_area(interpolate='monotone', opacity = 0.8).encode(
        alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
        alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="Immigration Rate", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
        alt.Y2('upper_hpd_linear_50' ),
    ).properties(
        width=450,
        height=150
    )

    band_mig = alt.Chart(mig_summary).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
        alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
        alt.Y('lower_hpd_linear',axis=alt.Axis(title="Immigration Rate", grid=False), scale=alt.Scale(domain=(0, 100))),
        alt.Y2('upper_hpd_linear' ),
    ).properties(
        width=450,
        height=150
    )

    mig_plot = (line_mig + band_mig)
    
    
    return ne_case_plot, mig_plot

In [80]:
folder_path = '../multitree_coalescent/results/sensitivity/'

# List all files in sims folder
log_files = [(folder_path + file) for file in os.listdir(folder_path) if file.endswith('.log')]

#put everything in order
log_files.sort()

In [81]:
log_files

['../multitree_coalescent/results/sensitivity/1-multicoal_la_01_22_GTRSubModel.log',
 '../multitree_coalescent/results/sensitivity/2-multicoal_la_01_22_8GammaCatergories.log',
 '../multitree_coalescent/results/sensitivity/3-multicoal_la_01_22_CasePriorNonSmoothedCases.log',
 '../multitree_coalescent/results/sensitivity/4-multicoal_la_01_22_CasePriorZeroCases.log',
 '../multitree_coalescent/results/sensitivity/5-multicoal_la_01_22_SkyGrowthPrior.log']

In [82]:
chart_list_ne = []
chart_list_mig = []

for file in log_files:
    base_filename = file.split("/")[3].split(".")[0]
    
    plot1, plot2 = plot_ne_and_intro_comparison(file)
    plot1 = plot1.properties(title="Ne vs cases:" +  "_"+ file.split("_")[-1].split(".")[0])
    plot2 = plot2.properties(title="Rate of viral importation:"  + file.split("_")[-1].split(".")[0])

    
    chart_list_ne.append(plot1)
    chart_list_mig.append(plot2)

5957
4170
5957
4170
5271
3690
5271
3690
6067
4247
6067
4247
6098
4269
6098
4269
5001
3501
5001
3501


In [83]:
plots_1 = alt.vconcat(*chart_list_ne)
plots_2 = alt.vconcat(*chart_list_mig)

In [84]:
plots_1 | plots_2

alt.HConcatChart(...)